## 4-1. 들어가며

### 목차

    1. 시퀀스? 스퀀스!
    2. I 다음 am을 쓰면 반 이상은 맞더라
    3. 실습
        1) 데이터 다듬기
        2) 인공지능 학습시키기
        3) 잘 만들어졌는지 평가하기
    4. 프로젝트 : 멋진 작사가 만들기

## 4-2. 시퀀스? 스퀀스!

시퀀스 데이터란 나열된 데이터를 의미하므로,

각 요소들이 동일한 속성을 띌 필요가 없으며,

어떤 기준에 따라 정렬 되어 있지 않아도 됩니다.

## 4-3. I 다음 am을 쓰면 반 이상은 맞더라

인공지능이 글을 이해하게 하는 방식도 위와 같습니다.

어떤 문법적인 원리를 통해서가 아니고, 수많은 글을 읽게 함으로써

나는 , 밥을, 그다음이 먹는다 라는 사실을 알게 하는 거죠.

그런 이유에서 많은 데이터가 곧 좋은 결과를 만들어냅니다.

단어를 적재적소에 활용하는 능력이 발달된다고 할까요?

이 방식을 가장 잘 처리하는 인공지능 중 하나가 순환신경망(RNN) 입니다.

이번 시간엔 자세한 내용보다는 간단한 구조를 중심으로 공부하겠습니다.

In [1]:
# 순환신경망 (RNN) 으로 AI가 문장을 만드는 메커니즘으 파이썬으로 구현한 것.

sentence = " 나는 밥을 먹었다 "

source_sentence = "<start>" + sentence
target_sentence = sentence + "<end>"

print("Source 문장:", source_sentence)
print("Target 문장:", target_sentence)

Source 문장: <start> 나는 밥을 먹었다 
Target 문장:  나는 밥을 먹었다 <end>


### 언어 모델 (Language Model)

n−1개의 단어 시퀀스 w1, ... , w_{n-1} 이 주어졌을 때,

n 번째 단어 w_n 으로 무엇이 올지를 예측하는 확률 모델을

언어 모델(Language Model) 이라고 부릅니다.

파라미터 θ로 모델링 하는 언어 모델을 다음과 같이 표현할 수 있습니다.

P(w_n | w_1, ..., w_{n-1};θ)

학습된 언어 모델을 학습 모드가 아닌 테스트 모드로 가동하면,

그 모델은 일정한 단어 시퀀스가 주어질 때, 다음 단어, 그 다음 단어를 계속해서 예측해 낼 것입니다.

즉, 잘 학습된 언어 모델은 훌륭한 문장 생성기로 동작하게 됩니다.


## 4-4. 실습 (1) 데이터 다듬기

### 데이터 다운로드

클라우드 환경에서는 데이터를 준비해두었으니 Cloud shell에서 아래 명령어를 입력해 연결만 시켜주세요.

 mkdir -p ~/aiffel/lyricist/models
 
 ln -s ~/data ~/aiffel/lyricist/data

In [2]:
# 준비한 파일의 내용 확인

import os, re
import numpy as np
import tensorflow as tf

# 파일을 읽기모드로 열고
# 라인 단위로 끊어서 list 형태로 읽어옵니다.
file_path = os.getenv('HOME') + '/aiffel/lyricist/data/shakespeare.txt'
with open(file_path, "r") as f:
    raw_corpus = f.read().splitlines()

# 앞에서부터 10라인만 화면에 출력해 볼까요?
print(raw_corpus[:9])

['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.', '', 'First Citizen:', 'You are all resolved rather to die than to famish?', '']


In [4]:
# 공백인 문장은 길이를 검사하여 길이가 0이라면 제외를 시켜 봅니다.

for idx, sentence in enumerate(raw_corpus): # enumerate 열거하다 라는 뜻으로, 리스트가 있는 경우 순서와 리스트의 값을 전달하는 기능
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

Before we proceed any further, hear me speak.
Speak, speak.
You are all resolved rather to die than to famish?


전처리를 위해 정규표현식(Regex)을 이용한 필터링이 유용하게 사용됩니다.

In [6]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [7]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> before we proceed any further , hear me speak . <end>',
 '<start> speak , speak . <end>',
 '<start> you are all resolved rather to die than to famish ? <end>',
 '<start> resolved . resolved . <end>',
 '<start> first , you know caius marcius is chief enemy to the people . <end>',
 '<start> we know t , we know t . <end>',
 '<start> let us kill him , and we ll have corn at our own price . <end>',
 '<start> is t a verdict ? <end>',
 '<start> no more talking on t let it be done away , away ! <end>',
 '<start> one word , good citizens . <end>']

In [8]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  143   40 ...    0    0    0]
 [   2  110    4 ...    0    0    0]
 [   2   11   50 ...    0    0    0]
 ...
 [   2  149 4553 ...    0    0    0]
 [   2   34   71 ...    0    0    0]
 [   2  945   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fd89f0e4210>


In [10]:
# 생성된 텐서 데이터를 3번째 행, 10번째 열까지만 출력

print(tensor[:3, :10])

[[   2  143   40  933  140  591    4  124   24  110]
 [   2  110    4  110    5    3    0    0    0    0]
 [   2   11   50   43 1201  316    9  201   74    9]]


In [20]:
# tokenizer에 구축된 단어 사전의 인덱스

for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : .
6 : the
7 : and
8 : i
9 : to
10 : of


In [12]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2 143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0
   0   0]
[143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0   0
   0   0]


In [13]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 20), (256, 20)), types: (tf.int32, tf.int32)>

# 데이터셋을 생성하기 위해 거쳐 온 과정

(1) 정규표현식을 이용한 corpus 생성

(2) tf.keras.preprocessing.text.Tokenizer를 이용해 corpus를 텐서로 변환

(3) tf.data.Dataset.from_tensor_slices()를 이용해 corpus 텐서를 tf.data.Dataset객체로 변환

dataset을 얻음으로써 데이터 다듬기 과정은 끝났습니다.

tf.data.Dataset에서 제공하는 shuffle(), batch() 등

다양한 데이터셋 관련 기능을 손쉽게 이용할 수 있게 되었군요.

이 모든 일련의 과정을 텐서플로우에서의 데이터 전처리 라 칭합니다!

## 4-5. 실습 (2) 인공지능 학습시키기

Embedding : 단어를 추상적으로 변환하는 역할
    
RNN 1 (LSTM) & RNN 2 (LSTM) : 문장을 순차적으로 읽으며 단어 간의 연관성을 분석
    
Linear (Dense) :RMM 에서 만들어낸 결과를 바탕으로 생성할 단어를 결정

In [14]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size): 
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

embedding_size는 워드 벡터의 차원수,

즉 단어가 추상적으로 표현되는 크기 입니다.

다시 말해서, 값이 커질수록 단어의 추상적인 특징들을 더 잡아낼 수 있지만,

그만큼 충분한 데이터가 주어지지 않으면 오히려 혼란만을 야기할 수 있습니다.

LSTM 레이어의 hidden state 의 차원수인 hidden_size 도 같은 맥락입니다.



hidden_size 는 모델에 얼마나 많은 일꾼을 둘 것인가? 로 이해해도 크게 엇나가지 않습니다.

그 일꾼들은 모두 같은 데이터를 보고 각자의 생각을 가지는데,

역시 충분한 데이터가 주어지면 올바른 결정을 내리겠지만 그렇지 않으면 배가 산으로 갈 뿐 입니다. 

In [15]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 20, 7001), dtype=float32, numpy=
array([[[-2.63612892e-04, -2.05965043e-05,  1.55494723e-04, ...,
          9.99320255e-05,  2.43276474e-04,  1.37046300e-04],
        [-4.40024305e-04, -6.77920470e-05,  4.19543881e-04, ...,
          1.05223029e-04,  6.03980618e-04,  1.49964239e-04],
        [-4.56051261e-04, -4.35196962e-05,  8.03412055e-04, ...,
          1.16965202e-04,  9.13601893e-04,  4.56084439e-04],
        ...,
        [-3.60699894e-04,  3.47318535e-04, -1.61137577e-04, ...,
         -7.33869325e-04, -1.95076654e-03,  3.00926360e-04],
        [-4.09587432e-04,  8.14406667e-04, -1.82715448e-04, ...,
         -6.68100547e-04, -2.14359793e-03,  6.49008318e-04],
        [-4.43089259e-04,  1.23483874e-03, -1.61892254e-04, ...,
         -6.51027542e-04, -2.24090274e-03,  9.74428898e-04]],

       [[-2.63612892e-04, -2.05965043e-05,  1.55494723e-04, ...,
          9.99320255e-05,  2.43276474e-04,  1.37046300e-04],
        [-2.90818949e-04,  3.88097578e-05,  2.

모델의 최종 출력 텐서 shape를 유심히 보면 shape=(256, 20, 7001)임을 알 수 있습니다.

7001은 Dense 레이어의 출력 차원수입니다.

7001개의 단어 중 어느 단어의 확률이 가장 높을지를 모델링해야 하기 때문입니다.

256은 이전 스텝에서 지정한 배치 사이즈입니다.

dataset.take(1)를 통해서 1개의 배치, 즉 256개의 문장 데이터를 가져온 것입니다.

그렇다면 20은 무엇을 의미할까요?

비밀은 바로 tf.keras.layers.LSTM(hidden_size, return_sequences=True)로 호출한

LSTM 레이어에서 return_sequences=True이라고 지정한 부분에 있습니다.

즉, LSTM은 자신에게 입력된 시퀀스의 길이만큼 동일한 길이의 시퀀스를 출력한다는 의미입니다.

만약 return_sequences=False였다면 LSTM 레이어는 1개의 벡터만 출력했을 것입니다.

그런데 문제는, 우리의 모델은 입력 데이터의 시퀀스 길이가 얼마인지 모른다는 점입니다.

모델을 만들면서 알려준 적도 없습니다. 그럼 20은 언제 알게된 것일까요?

네, 그렇습니다. 데이터를 입력받으면서 비로소 알게 된 것입니다.

우리 데이터셋의 max_len이 20으로 맞춰져 있었던 것입니다.

In [16]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  1792256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  7176025   
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


In [17]:
# optimizer와 loss등은 차차 배웁니다
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
93/93 [==============================] - 65s 671ms/step - loss: 4.4028
Epoch 2/30
93/93 [==============================] - 63s 677ms/step - loss: 2.8139
Epoch 3/30
93/93 [==============================] - 63s 680ms/step - loss: 2.7305
Epoch 4/30
93/93 [==============================] - 63s 679ms/step - loss: 2.6153
Epoch 5/30
93/93 [==============================] - 63s 680ms/step - loss: 2.5398
Epoch 6/30
93/93 [==============================] - 63s 680ms/step - loss: 2.4933
Epoch 7/30
93/93 [==============================] - 63s 680ms/step - loss: 2.4395
Epoch 8/30
93/93 [==============================] - 63s 679ms/step - loss: 2.3871
Epoch 9/30
93/93 [==============================] - 63s 680ms/step - loss: 2.3322
Epoch 10/30
93/93 [==============================] - 63s 680ms/step - loss: 2.2897
Epoch 11/30
93/93 [==============================] - 63s 679ms/step - loss: 2.2428
Epoch 12/30
93/93 [==============================] - 63s 679ms/step - loss: 2.1872
Epoch 13/30
9

## 4-6. 실습 (3) 잘 만들어졌는지 평가하기

모델이 작문을 잘하는지 컴퓨터 알고리즘이 평가하는 것은 무리가 있습니다.

만약에 그게 가능했다면 우리가 지금껏 해온 독후감 숙제를 컴퓨터가 채점했겠죠?

따라서 작문 모델을 평가하는 가장 확실한 방법은 작문을 시켜보고 직접 평가하는 겁니다.

아래 generate_text 함수는 모델에게 시작 문장을 전달하면

모델이 시작 문장을 바탕으로 작문을 진행하게 합니다.

In [18]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

generate_text() 함수에서 init_sentence를 인자로 받고는 있습니다.

이렇게 받은 인자를 일단 텐서로 만들고 있습니다.

디폴트로는 <start> 단어 하나만 받는군요.

while의 첫 번째 루프에서 test_tensor에 <start> 하나만 들어갔다고 합시다.
    
우리의 모델이 출력으로 7001개의 단어 중 A를 골랐다고 합시다.
    
while의 두 번째 루프에서 test_tensor에는 <start> A가 들어갑니다.
    
그래서 우리의 모델이 그다음 B를 골랐다고 합시다.
  
while의 세 번째 루프에서 test_tensor에는 <start> A B가 들어갑니다.
    
그래서..... (이하 후략)

In [19]:
# 위 문장 생성 함수를 실행

generate_text(model, tokenizer, init_sentence="<start> he")

'<start> he hath got a <unk> of a <unk> <end> '

지금은 짧은 한 줄짜리 문장을 생성하지만,

더 공부하시면 최고의 작문 모델 GPT-2를 이용해 더욱 멋진 문장 생성 프로젝트를 할 수 있을 것입니다.